In [1]:
# !pip install folium
# !pip install pandas
# !pip install numpy
# !pip install ipywidgets

     |████████████████████████████████| 94 kB 2.6 MB/s eta 0:00:011


In [1]:
#%matplotlib widget
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
import folium
import json

In [2]:
# widgets
import ipywidgets as widgets

Предсказания загружу из 6 недели, чтобы не пересчитывать.

In [3]:
data = pd.read_csv('NYC_2014-2016.csv', index_col=0)
data_may = data[:-720]
print(data_may.shape)
data_may.tail()

(21168, 102)


,1075,1076,1077,1125,1126,1127,1128,1129,1130,1131,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
2016-05-31 19:00:00,106,167,171,88,293,401,433,284,428,222,...,1,5,9,347,304,104,15,172,17,103
2016-05-31 20:00:00,99,166,165,65,301,350,404,351,444,190,...,0,16,10,336,271,154,23,284,116,111
2016-05-31 21:00:00,84,173,107,78,256,340,416,426,493,173,...,0,37,5,289,306,167,26,212,84,130
2016-05-31 22:00:00,83,98,127,94,174,345,415,411,490,144,...,1,20,9,392,299,123,12,222,53,123
2016-05-31 23:00:00,34,77,58,53,118,211,331,338,434,121,...,2,12,10,430,153,157,17,192,77,132


In [4]:
# Open predicted data
pred_june = np.zeros((6, 715, 102))
for l in range(1, 7):
    name_rf = 'predict_june_rf_{}.csv'.format(l)
    pred_june[l-1, :, :] = np.genfromtxt(name_rf, delimiter=',')

In [5]:
# Open GeoJSON
with open('NYC_sq_mean.geojson') as f:
    geojs_mean = json.load(f)

In [6]:
import matplotlib.pyplot as plt
from folium import plugins
from ipywidgets import Layout, interact_manual
from folium.features import GeoJson, GeoJsonTooltip, GeoJsonPopup

In [7]:
style = {'description_width': 'initial'}

slider_lag = widgets.IntSlider(
    value=1,
    min=1,
    max=6,
    step=1,
    description='Time lag',
    style=style)

slider_lag.style.handle_color = 'lightblue'

region = widgets.SelectMultiple(
    options = data.columns.tolist(),
    description='Region',
    disabled=False,
    style=style,
    layout = Layout(width='80%', height='180px')
)

In [ ]:
# Create interactive version of function with DatePickers
interact(stats_for_article_published_between,
        start_date=widgets.DatePicker(value=pd.to_datetime('2018-01-01')),
        end_date=widgets.DatePicker(value=pd.to_datetime('2019-01-01')))

In [10]:
@interact_manual(lag=slider_lag, regions=region)
def create_map(lag=1, regions='1075', start_time):
    # Create real data
    data = pd.read_csv('NYC_2014-2016.csv', index_col=0)
    real_data = data[regions[0]].shift(-lag)
    # Create table for calculate mean
    mn = np.mean(pred_june[lag-1, :, :], axis=0)
    reg = data.columns
    
    table = pd.DataFrame([reg, mn]).T
    table.columns = ['region', 'trip_mean']
    # Create start Map
    NYC = folium.Map(location=[40.733, -73.9], zoom_start=11)

    # Add layers for Popup and Tooltips
    popup = GeoJsonPopup(
                    fields=['region'],
                    aliases=['Region'], 
                    localize=True,
                    labels=True,
                    style="background-color: yellow;")
    
    tooltip = GeoJsonTooltip(
                    fields=['region'],
                    aliases=['Region'],
                    localize=True,
                    sticky=False,
                    labels=True,
                    style="""background-color: #F0EFEF;
                             border: 1px solid black;
                             border-radius: 3px;
                             box-shadow: 3px;""",
                    max_width=800)
    
    # Add choropleth layer
    g = folium.Choropleth(geo_data=geojs_mean, 
                          data=table, 
                          columns=['region', 'trip_mean'],
                          key_on="feature.id", 
                          fill_color='YlGn', fill_opacity=0.7, 
                          line_opacity=0.4, 
                          legend_name='Mean value of trip in June', 
                          highlight=True).add_to(NYC)

    
    folium.GeoJson(geojs_mean, 
                   style_function=lambda feature: { 'fillColor': '#ffff00',
                                                   'color': 'black',
                                                   'weight': 0.2,
                                                   'dashArray': '5, 5'},
                    tooltip=tooltip,
                    popup=popup).add_to(NYC)
    
    folium.LayerControl().add_to(NYC)
    display(NYC)
    # Predict and real values
    per = pd.date_range(start='5/31/2016', end='7/1/2016', freq='H', closed='left')
    period = per[23:-6]
    region_1 = region.index[0]
    
    plt.figure(figsize=(20,10))
    plt.subplot(212)
    plt.plot(period, pred_june[lag-1, :, region_1].reshape(715,), label='Predict NY Taxi in June')
    plt.plot(period, real_data[-721:-6].values, label='Real values NY Taxi in June')
    plt.legend()
    plt.title('Predicted values in region: {} , with lag {}'.format(regions[0], lag));

interactive(children=(IntSlider(value=1, description='Time lag', max=6, min=1, style=SliderStyle(description_w…

Представление немного скучноватое, потому что Jupyter Notebook не показывает интерактивные карты bqplot и ipyleaflet на моём компьютере, а folium не поддерживает виждеты. Возможно это связано с тем, что я использую Google Chrome. В репозитарии лежит код, его можно открыть через Jupyter Lab.

 Пробовал запустить Jupyter Lab, выдаёт ошибку